# GREGoR Data Tracking and Reporting
__author: DCC__ <br>
__last edited: 07/05/2023__ <br>

In [ ]:
# import modules
import os
import io
import pandas as pd
import terra_pandas as tp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
import seaborn as sns
from functools import reduce

In [ ]:
project = os.environ['WORKSPACE_NAMESPACE']
workspace = os.environ['WORKSPACE_NAME']
bucket = os.environ['WORKSPACE_BUCKET'] + "/" 

#print("Current GREGoR upload cycle: U02")
print("Terra Billing project: " + project)
print("Workspace: " + workspace)
print("Workspace storage bucket: " + bucket)

In [ ]:
bucket_size = pd.read_table('bucket_size', header = None, sep = ' ')
bucket_size.columns = ['WORKSPACE', 'SIZE', 'UNIT']

new_bucket_size = [] 
for index, rows in bucket_size.iterrows(): 
    # print(rows['UNIT'])
    # print(rows['SIZE'])
    
    if rows['UNIT'] == 'GiB': 
        contib = rows['SIZE']/1000
        #print(contib)
        new_bucket_size.append(contib)
    elif rows['UNIT'] == 'TiB':
        tib = rows['SIZE']
        new_bucket_size.append(tib)

print('Total size of GREGoR U02 workspace buckets is: ' + str(sum(new_bucket_size)) + ' TiB')

## Overview

The GREGoR Data Tracking Report provides summaries of the latest data contributed to the GREGoR Consortium by member Research Centers (RCs). Information regarding data generation and data deposited to AnVIL are derived from the RC quarterly reports and the combined consortium workspaces. Graphical and tabular summaries of participant, experiment and phenotype information are generated from metadata provided in the GREGoR data model.  

In [ ]:
current_upload_cycle = 'U02'

__Latest upload cycle:__ {{ current_upload_cycle }} 

In [ ]:
# function(s) for reading in RC reporting

def readGoogleSheet(url):
    gsheet_url = url
    csv_export_url = gsheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
    gsheet_df = pd.read_csv(csv_export_url)
    return gsheet_df

# function(s) for AnVIL data tables
def readDatatable(data_table, project, workspace): 
    new_table = tp.table_to_dataframe(data_table, workspace_namespace=project, workspace=workspace)
    return new_table

def formatIndex(df):
    mylist = []
    for i in df.index:
        if type(i) == tuple: 
            str = '_'.join(i)
            mylist.append(str)
        else: 
            mylist.append(i)
    df.index = mylist
    df.index = df.index.str.upper()
    return df

def summarizeData(df, column):
    df['participant_id'] = df.index
    df = pd.DataFrame(df.groupby(by=[column], dropna=False)['participant_id'].count())
    df.columns = ['NO. OF PARTICIPANTS']
    formatIndex(df)
    df.columns = df.columns.str.upper()
    return df

def participantSummary(df):
    df['participant_id'] = df.index
    df = df.groupby(by=['gregor_center', 'consent_code'], as_index = True)[['participant_id', 'family_id']].nunique()
    df.loc["Total"] = df.sum()
    df.columns = ['participants', 'families']
    formatIndex(df)
    df.columns = df.columns.str.upper()
    return df

def experimentMerge(participant_df, analyte_df, experiment_df):
    df = pd.merge(participant_df, analyte_df, left_index = True, right_on='participant_id')
    df1 = pd.merge(df, experiment_df, left_index = True, right_on='analyte_id')
    return df1

def experimentSummary(participant_df, analyte_df, experiment_df):
    df1 = experimentMerge(participant_df, analyte_df, experiment_df)
    experiment_type = df1.groupby(by=['gregor_center','experiment_type'])['participant_id'].nunique().unstack()
    experiment_type = experiment_type.fillna(0).astype(int)
    experiment_type.loc["Total"] = experiment_type.sum()
    formatIndex(experiment_type)
    experiment_type.columns = experiment_type.columns.str.upper()
    experiment_type.columns.name = None
    return experiment_type

def phenotypeMerge(participant_df, phenotype):
    participant_terms = pd.DataFrame(phenotype.groupby('participant_id')['term_id'].count())
    participant_terms.index.name = None
    df = pd.merge(participant_df, participant_terms, left_index = True, right_index = True)
    return df


In [ ]:
# read in combined google tracking sheets
# GREGoR combined RC reporting sheets for U1
summary_report_U1 = readGoogleSheet("https://docs.google.com/spreadsheets/d/1EtOc6ggsJN6QZ4VuqCmBvUIHplMDWxshQlYnALDCHrs/edit#gid=0")

# GREGoR combined RC reporting sheets for U2
summary_report_U2 = readGoogleSheet("https://docs.google.com/spreadsheets/d/1ndQRPsJW6d8kWIq2j9JHui9-sudrtcyDoUZ_qX821X8/edit#gid=0")

In [ ]:
# read in AnVIL tables
participant = readDatatable('participant', project, workspace)
family = readDatatable('family', project, workspace)
phenotype = readDatatable('phenotype', project, workspace)
analyte = readDatatable('analyte', project, workspace)
experiment_dna_short_read = readDatatable('experiment_dna_short_read', project, workspace, )
aligned_dna_short_read = readDatatable('aligned_dna_short_read', project, workspace)

__Abbreviations:__<br> 
__RCs:__ BCM = Baylor College of Medicine Research Center; BROAD = Broad Institute; CNH_I = Children's National Hospital/Invitae; GSS = GREGoR Stanford Site; UW-CRDR = University of Washington Center for Rare Disease Research.
BCM = Baylor College of Medicine Research Center; BROAD = Broad Institute; CNH_I = Children's National Hospital/Invitae; GSS = GREGoR Stanford Site; UW-CRDR = University of Washington Center for Rare Disease Research; <br>
__Consent codes:__ GRU = General research use and clinical care; HMB = Health/medical/biomedical research and clinical care

## Participants and Families

The section below includes summaries of the participants, families, and family relationships in the GREGoR Combined Dataset. 

__Table 1. The number of participants and families in the GREGoR Combined Consortium Dataset (upload cycle: {{current_upload_cycle}}).__ 

In [ ]:
participant_data = participantSummary(participant)
participant_data.index.name = 'GREGoR_CENTER'
participant_data

In [ ]:
family_size = summarizeData(participant, 'family_id')
family_size.columns = ['FAMILY SIZE']
plt.figure(figsize=(7,5))

sns.set(style="white", font='sans-serif', font_scale=1.2)

sns.histplot(data = family_size, x = 'FAMILY SIZE', color='#076839', edgecolor = 'black', discrete= True)
plt.ylabel('SAMPLE COUNT')
#plt.title('Distribution of Family Size in the GREGoR Combined Consortium Dataset')

plt.show()

__Figure 1.__ The distribution of family size in the GREGoR Combined Consortium Dataset.

__Table 2. The number of participants by their relationship to the proband.__ 

In [ ]:
proband_rel_data = summarizeData(participant, 'proband_relationship')
proband_rel_data.index.name = 'PROBAND_RELATIONSHIP'
proband_rel_data

In [ ]:
# plot pie chart
df = proband_rel_data.loc[proband_rel_data['NO. OF PARTICIPANTS'] > 7]
df = df.replace(22, 48)
labels = df.index.str.lower()

color = ['#076839', '#388660', '#519574', '#6aa488', '#9bc2af']

plt.figure(figsize=(7,5))
plt.pie(df['NO. OF PARTICIPANTS'], labels = labels, colors = color, autopct='%.2f', labeldistance=1.25, 
        radius=1.25, 
        textprops={'fontsize': 15, 'fontname' : 'serif', 'ha' : 'center' , 'color' : 'black'},
        wedgeprops={ 'linewidth' : 1.5, 'edgecolor' : "white" })
plt.tight_layout()
plt.plot()

__Figure 2.__ Pie chart showing the percentage of proband relationships in the GREGoR Combined Consortium Dataset. Note proband relationship categories were collapsed into 'other' if the count < 10; Self = proband.   

__Table 3. The number of female and male participants in the GREGoR Combined Consortium Dataset (current_upload cycle: {{current_upload_cycle}}).__

In [ ]:
sex = summarizeData(participant, 'sex')
sex

## Sample Processing and Experiment Summaries 

In [ ]:
frames = [summary_report_U1, summary_report_U2]
df = pd.concat(frames, axis = 0)

In [ ]:
plt.figure(figsize=(14, 10))
plt.subplots_adjust(hspace=0.5)

#plt.suptitle("Summary of Research Center Sample/Sequencing Reports")
sns.set(style="white", font='sans-serif', font_scale=1.2)

rc_list = ['BCM', 'BROAD', 'CNH_I', 'GSS', 'UW_CRDR']

for n, rc in enumerate(rc_list):
    # print(rc)
    # add a new subplot iteratively
    ax = plt.subplot(3, 2, n + 1)
    single_rc = df[df["GREGoR_CENTER"] == rc]
    single_rc_sub =  single_rc[['SAMPLES_PREPARED', 'NUM_OF_SAMPLES_COMPLETED', 
                                      'SAMPLE_FILES_UPLOADED_TO_ANVIL']]
    single_rc_sub.columns = ['PREPARED', 'SEQUENCED', 'UPLOADED TO AnVIL']
    single_rc_sub.index = ['U1', 'U2']
    single_rc_sub_t = single_rc_sub.transpose()
    #print(single_rc_sub_t)
    
    sns.barplot(ax = ax, data = single_rc_sub_t, x =  single_rc_sub_t.index, y = "U2", color = "white", edgecolor = 'black')
    sns.barplot(ax = ax, data= single_rc_sub_t, x = single_rc_sub_t.index, y= "U1", estimator=sum,  color='#076839', edgecolor = 'black')
    plt.title(rc)
    plt.xlabel = None
    ax.set_ylabel('NO. OF SAMPLES')
    #plt.xticks(rotation=45)
    
    
    
# legend
top_bar = mpatches.Patch(color='white', label='U2')
bottom_bar = mpatches.Patch(color='#076839', label='U1')
plt.legend(handles=[top_bar, bottom_bar], facecolor = "lightgray", loc = 'lower right', 
           bbox_to_anchor=(0.5, 0.4, 1, 0.5), fontsize = 'large', shadow = True)
plt.show()

__Figure 3.__ Quarterly report completed by the GREGoR RCs to track short read DNA experiments. Stacked bar charts show the number of samples prepared, the number of samples that completed sequencing and the number of samples uploaded to AnVIL. A stacked bar chart show sample summaries for each RC provided for U01 (green) and U02 (white).  

__Table 4. Summary of RC quarterly reports for short read DNA experiments in the {{current_upload_cycle}} cycle.__<br> 

In [ ]:
summary_report_U2_sub = summary_report_U2[['SAMPLES_PREPARED', 
                                           'NUM_SAMPLES_IN_SEQ', 'NUM_OF_SAMPLES_COMPLETED', 'NUM_FAILURES', 'SAMPLE_FILES_UPLOADED_TO_ANVIL']]
summary_report_U2_sub.index = summary_report_U2['GREGoR_CENTER']


In [ ]:
summary_report_U2_sub

__Table 5. Summary of RC quarterly reports for short read RNA experiments in the {{current_upload_cycle}} cycle.__<br> 

In [ ]:
#add RNA table

__Table 6. The number of exomes and genomes in the  GREGoR Combined Consortium Dataset.__ <br>
_Note: These numbers are derived from the experiment_dna_short_read table._ 

In [ ]:
experiment_type_by_center = experimentSummary(participant, analyte, experiment_dna_short_read)
experiment_type_by_center.index.name = 'GREGoR_CENTER'
experiment_type_by_center

__Table 7. The number of aligned sequencing files (i.e. BAMS or CRAMs) in the GREGoR Combined Consortium Dataset.__ <br> 
_Note: These numbers are derived from the 'aligned_dna_short_read' data table._

In [ ]:
df = experimentMerge(participant, analyte, experiment_dna_short_read)
participant_aligned = pd.merge(df, aligned_dna_short_read, left_index = True, right_on='experiment_dna_short_read_id')
aligned_files_by_center = participant_aligned.groupby(by=['gregor_center'])[['aligned_dna_short_read_file']].nunique()
aligned_files_by_center.loc["Total"] = aligned_files_by_center.sum()
aligned_files_by_center.columns = ['No. of short read DNA files']
    
formatIndex(aligned_files_by_center)
aligned_files_by_center.columns = aligned_files_by_center.columns.str.upper()
aligned_files_by_center.index.name = 'GREGoR_CENTER'
aligned_files_by_center

## Phenotype Data

__Table 8. Summary of 'affected status' in the GREGoR Combined Consortium dataset__

In [ ]:
affected_data = summarizeData(participant, 'affected_status')
affected_data.loc['TOTAL'] = affected_data.sum()
affected_data.index.name = 'AFFECTED_STATUS'
affected_data

__Summary of phenotype ontology terms in the GREGoR Combined Consortium Dataset__

In [ ]:
unique_phenotypes = phenotype[~phenotype['term_id'].duplicated()]
participant_terms = pd.merge(participant,phenotype, left_index = True, right_on = 'participant_id')

#unique_phenotypes
print('Number of phenotype terms: ' + str(len(phenotype)))
print('Number of unique phenotype terms: ' + str(len(unique_phenotypes)))
print('Number of participants with phenotype terms: ' + str(participant_terms['participant_id'].nunique()))

__Table 9. Common phenotype ontology terms in the GREGoR Combined Consortium dataset__

In [ ]:
term_count = pd.DataFrame(phenotype.groupby('term_id', dropna=False)['participant_id'].count())
term_count_sorted = term_count.sort_values('participant_id', ascending=False)
term_count_sorted["term"] = term_count_sorted.index
term_count_sorted.reset_index(drop=True, inplace = True)
term_name = ['Global developmental delay', 'Seizure', 'Intellectual disability', 'Hypotonia', 
             'Morphological central nervous system abnormality','Bicuspid aortic valve', 
             'Thoracic aortic aneurysm', 'Abnormal cerebral cortex morphology', 'Microcephaly', 'Muscle weakness']
term_name = pd.DataFrame(term_name)

term_count_sorted_top10 = term_count_sorted[:10]
frames = [term_count_sorted_top10, term_name]
term_count_sorted_top10 = pd.concat(frames, axis = 1)
term_count_sorted_top10 = term_count_sorted_top10[['term', 0, 'participant_id']]

term_count_sorted_top10.columns = ['term_id', 'term_name', 'no. of participants']
term_count_sorted_top10.columns = term_count_sorted_top10.columns.str.upper()
term_count_sorted_top10

In [ ]:
term_count_singletons = term_count[term_count['participant_id'] == 1]
print('Note: A total of ' + str(term_count_singletons.shape[0]) + ' terms occur only once in the dataset')

## Summary of Data Completeness 

This section summarizes participants with genotype and phenotype information in the GREGoR Combined Dataset. Phenotype ontology terms are primarily expected for probands and affected relatives.

In [ ]:
# do relevant subsetting
probands = participant[participant['proband_relationship'] == 'Self']
other_affecteds = participant[(participant['proband_relationship'] != 'Self') & (participant['affected_status'] == 'Affected')] # other affecteds that are not probands
unaffecteds = participant[(participant['proband_relationship'] != 'Self') & (participant['affected_status'] == 'Unaffected')]

proband_terms = phenotypeMerge(probands, phenotype)
other_affected_terms = phenotypeMerge(other_affecteds, phenotype)
unaffected_terms = phenotypeMerge(unaffecteds, phenotype)

seq_proband = pd.merge(probands, participant_aligned, left_index = True, right_on = 'participant_id') # sequenced probands with HPO terms
seq_affected = pd.merge(other_affecteds, participant_aligned, left_index = True, right_on = 'participant_id') # sequenced probands with HPO terms
seq_unaffected = pd.merge(unaffecteds, participant_aligned, left_index = True, right_on = 'participant_id') 


seq_proband_terms = pd.merge(proband_terms, participant_aligned, left_on = 'participant_id', right_on = 'participant_id') 
seq_affected_terms = pd.merge(other_affected_terms, participant_aligned, left_index = True, right_on = 'participant_id') 
seq_unaffected_terms = pd.merge(unaffected_terms, participant_aligned, left_index = True, right_on = 'participant_id')

In [ ]:
datacomp_series = { 'TOTAL': [len(probands), len(other_affecteds), len(unaffecteds)], 
                'SEQUENCED' : [len(seq_proband), len(seq_affected), len(seq_unaffected)],
                'SEQUENCED_WITH_PHENOTYPE' : [len(seq_proband_terms), len(seq_affected_terms), 
                                              len(seq_unaffected_terms)]
               }

__Table 10. The number of 'aligned DNA short read files' and phenotype terms for probands, affected and unaffected relatives__

In [ ]:
datacomp_df = pd.DataFrame(datacomp_series)
datacomp_df.index = ['PROBANDS', 'OTHER_AFFECTED', 'UNAFFECTED']
datacomp_df